# 01. Tensor

In [ ]:
import torch

In [ ]:
# hyper params
dtype = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10
learning_rate = 1e-6

In [ ]:
# in/out data (might be placeholder)
# randomly generate in/out data with normal distribution
# numpy와 기본적으로 비슷하게 생겼다. pythonic
# dtype 나중에 꼬이면 골치아프니까 꼭 미리 지정해주자
# dim(x) = (N, D_in)
x = torch.randn(N, D_in).type(dtype) 
y = torch.randn(N, D_out).type(dtype)

In [ ]:
# Variables
# initialize weight matrix
W1 = torch.randn(D_in, H).type(dtype)
W2 = torch.randn(H, D_out).type(dtype)

In [ ]:
for i in range(500):
    # matrix multiplication => tensor_1.mm(tensor_2)
    hidden_layer = x.mm(W1)
    
    # relu activation(customize with torch.clamp)
    ## torch.clamp(input, min, max) => min_max 사이면 input 그대로, 그 외엔 min/max return
    hidden_layer_relu = hidden_layer.clamp(min=0)
    y_pred = hidden_layer_relu.mm(W2)
    
    # calculate loss function
    # euclidean distance(**2 써줘도 가능)
    loss = (y_pred - y).pow(2).sum()
    print(f"iter {i} loss: {loss}")
    
    # calculate gradient
    # 첫 예제니까 gradient 손 계산
    grad_y_pred = 2.0*(y_pred - y)
    grad_W2 = hidden_layer_relu.t().mm(grad_y_pred)
    grad_hidden_layer_relu = grad_y_pred.mm(W2.t())
    grad_hidden_layer = grad_hidden_layer_relu.clone()
    grad_hidden_layer[hidden_layer < 0] = 0 #이런 식으로 indexing도 가능
    grad_W1 = x.t().mm(grad_hidden_layer)
    
    W1 -= learning_rate * grad_W1
    W2 -= learning_rate * grad_W2
    

In [ ]:
# inference loss
((x.mm(W1).clamp(min=0).mm(W2) - y) ** 2).sum()

# 02. Autograd

 gradient 손 계산 탈출 - Variable 이용
 

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
# hyper params
dtype = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10
learning_rate = 1e-6

In [ ]:
# in/out data (might be placeholder)
# randomly generate in/out data with normal distribution
# Variable instance는 자동 gradient update 해줌
# requires_grad=False 하면 업뎃 제외
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

In [ ]:
# Variables
# initialize weight matrix
W1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
W2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

In [ ]:
for i in range(500):
    y_pred = x.mm(W1).clamp(min=0).mm(W2)
    
    # **2 해도 되지만 sum()까지 method chain 해주기 위해
    loss = (y_pred - y).pow(2).sum()
    print(f"iter {i} loss: {loss}")
    
    # gradient를 자동으로 계산해준다
    # Variable instance의 .grad field를 이용해서 gradient 값 접근 가능하다.
    loss.backward()
    
    # w1.data: tensor/ w1.grad: Variable/ w1.grad.data: tensor
    # tensor에 대해서 연산을 취해준다.
    # Variable은 값에 직접적인 접근이 안되는 instance로 일단 생각해두자
    # 프린드해보면, Variable은 requires_grad 까지 출력되는 Variable이다.
    W1.data -= learning_rate * W1.grad.data
    W2.data -= learning_rate * W2.grad.data
    
    # gradient descent가 끝났으니, gradient를 0으로 만들어준다.
    W1.grad.data.zero_()
    W2.grad.data.zero_()
    

In [ ]:
# inference loss
((x.mm(W1).clamp(min=0).mm(W2) - y) ** 2).sum()

# 3. define new autograd function

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
class CustomReLU(torch.autograd.Function):
    """
    autograd.Function 상속, 나만의 autograd 정의
    """
    @staticmethod
    def forward(ctx, input):
        """
        ctx는 autograd.Fucntion에 미리 정의된 field
        backpropagation을 위한 정보를 저장하는 context object
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        input 값에 대한 loss의 변화도(gradient)를 return한다.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [ ]:
# hyper params
dtype = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10
learning_rate = 1e-6

In [ ]:
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)
W1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
W2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)


In [ ]:
for i in range(500):
    # 상속받은 torch.autograd.Function의 apply method 적용(정확히 말하면 그 method가 적용된 instance인듯?
    relu = CustomReLU.apply
    
    # 위에서 직접 relu를 계산해준 것과 달리, 이젠 relu instance로 편하게 연산
    y_pred = relu(x.mm(W1)).mm(W2)
    
    # **2 해도 되지만 sum()까지 method chain 해주기 위해
    loss = (y_pred - y).pow(2).sum()
    print(f"iter {i} loss: {loss}")
    
    # gradient를 자동으로 계산해준다
    # Variable instance의 .grad field를 이용해서 gradient 값 접근 가능하다.
    loss.backward()
    
    # w1.data: tensor/ w1.grad: Variable/ w1.grad.data: tensor
    # tensor에 대해서 연산을 취해준다.
    # Variable은 값에 직접적인 접근이 안되는 instance로 일단 생각해두자
    # 프린드해보면, Variable은 requires_grad 까지 출력되는 Variable이다.
    W1.data -= learning_rate * W1.grad.data
    W2.data -= learning_rate * W2.grad.data
    
    # gradient descent가 끝났으니, gradient를 0으로 만들어준다.
    W1.grad.data.zero_()
    W2.grad.data.zero_()
    

In [ ]:
# inference loss
((relu(x.mm(W1)).mm(W2) - y) ** 2).sum()

# 4. Static Graph

- tensorflow는 연산 그래프를 한 번 정의한 후 동일한 그래프를 계속해서 실행한다. -> 정적
    - 미리 정의되어있는 것이기 때문에 최적화에 좋다.
    - 여러 개의 gpu에 그래프를 배포할 수도 있다.
    - 반복 loop는 그래프 안에서 정의되어야한다.
- pytorch는 각각의 순전파 단계에서 새로운 연산 그래프를 정의한다. -> 동적
    - 그래프를 즉석(on-the-fly)에서 작성하기 때문에 일반적인 명령형으로 각각의 입력에 따라 다른 계산을 수행할 수 있다.
- tensorflow 코드 예제이므로 넘어가도록 한다.

# 5. nn 모듈

- 매번 autograd를 이용하는 방식으로 직접 SGD를 정의할 수 없다. 미리 정의된 모듈을 활용하자.


In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
N, D_in, H, D_out = 64, 1000, 100, 10
dtype = torch.FloatTensor
learning_rate = 1e-4

In [ ]:
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

In [ ]:
# feed forward nn을 그냥 layer 연산 순서대로 나열
## ReLU같은 activation function은 그냥 중간에 method만 써준다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out)
)

In [ ]:
# 앞서 사용하던 MSE loss를 미리 정의된 것으로 쉽게 사용할 수 있다.
loss_fn = torch.nn.MSELoss(size_average=False)

In [ ]:
for i in range(500):
    # model이라는 Sequential instance에 x를 넣어주기만 하면 된다. 마치 method처럼 호출이 가능하다.
    # y_pred 역시 반환되는 variable이다.
    y_pred = model(x)
    
    # 마찬가지로 method처럼 호출이 가능하다. loss 역시 variabledㅣ다.
    loss = loss_fn(y_pred, y)
    print(f"iter {i} loss: {loss.data}")
    
    # gradient를 0으로 세팅한다.
    model.zero_grad()
    
    # model의 모든 학습 가능한 변수에 대하여 gradient 계산
    # nn.Sequential 안에 정의한 모든 layer의 변수는 기본적으로 requires_grad=True로 세팅 되는 것으로 보인다.
    loss.backward()
    
    # param은 (당연한 말이겠지만) Variable 객체일 것이므로 .grad.data로 tenser에 접근 가능하다.
    # model.parameters()는 generator 객체이다. 즉, itertools로 접근 가능하다.
    # [*model.parameters()]로 까보면 requires_grad = True인 Variable로 선언된 것을 확인 가능하다.
    # Variable name은 따로 정의되지 않은 것으로 보이는데, 이것은 지정해줘야할 것 같다.
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data
    
    